# Set Up

In [ ]:
import os
from pathlib import Path

!pip install -q gdown
!gdown 1zda8TQ7XktshAU0bK-WFZ6RGHbBRXNkJ
!unzip -q sec_10_q.zip

Downloading...
From: https://drive.google.com/uc?id=1zda8TQ7XktshAU0bK-WFZ6RGHbBRXNkJ
To: /content/sec_10_q.zip
100% 14.5M/14.5M [00:00<00:00, 118MB/s]


In [ ]:
!pip install -q qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install -q pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.6 MB/s eta 0:00:00


In [ ]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.7 MB/s eta 0:00:00


In [ ]:
import os
from pathlib import Path
import os
from pathlib import Path
import fitz  # PyMuPDF for PDF extraction
import pandas as pd
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

# Nama docset (boleh untuk identifikasi internal)
DOCSET_NAME = "SEC 10Q Filings"

# Penyesuaian path berdasarkan struktur kamu
BASE_DIR = Path(os.getcwd()) / "sec-10-q"
DOCS_DIR = BASE_DIR / "data" / "v1" / "docs"
RAW_QUESTIONS_DIR = BASE_DIR / "data" / "raw_questions"
QNA_DATA_PATH = BASE_DIR / "data" / "v1" / "qna_data.csv"
QUESTIONS_CSV = RAW_QUESTIONS_DIR / "questions.csv"
QUESTIONS_WITH_LLM_ANSWERS_CSV = RAW_QUESTIONS_DIR / "questions_with_LLM_answers.csv"

# Buat folder jika belum ada
os.makedirs(DOCS_DIR, exist_ok=True)
os.makedirs(RAW_QUESTIONS_DIR, exist_ok=True)


# Daftar nama file PDF
FILE_NAMES = [f.name for f in DOCS_DIR.glob("*.pdf")]

assert len(FILE_NAMES) >= 6, "Please provide at least 6 files"

In [ ]:
!pip install -q langchain-community
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

LOCAL_LLM_CACHE_DB_FILE = os.environ.get("LOCAL_LLM_CACHE", "/tmp/docugami/.langchain.db")
os.makedirs(Path(LOCAL_LLM_CACHE_DB_FILE).parent, exist_ok=True)
set_llm_cache(SQLiteCache(database_path=LOCAL_LLM_CACHE_DB_FILE))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00


## Chunking

In [ ]:
doc_texts = {}
for pdf_file in DOCS_DIR.glob("*.pdf"):
    doc = fitz.open(pdf_file)
    full_text = "".join([page.get_text() for page in doc])
    doc_texts[pdf_file.name] = full_text
print(f"Loaded {len(doc_texts)} documents for processing.")

Loaded 20 documents for processing.


In [ ]:
# Chunking Function
def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    idx = 0
    while idx < len(words):
        chunk = words[idx: idx + chunk_size]
        chunks.append(" ".join(chunk))
        idx += chunk_size - overlap
    return chunks

In [ ]:
# Create DataFrame of Chunks
rows = []
for doc_name, text in doc_texts.items():
    for chunk_id, chunk in enumerate(chunk_text(text)):
        rows.append({"doc_name": doc_name, "chunk_id": chunk_id, "text": chunk})

df_chunks = pd.DataFrame(rows)
print(f"Created {len(df_chunks)} text chunks.")

df_chunks = pd.DataFrame(rows)
print(f"Created {len(df_chunks)} text chunks.")

Created 1067 text chunks.
Created 1067 text chunks.


In [ ]:
for name, text in doc_texts.items():
    print(f"{name} → {len(text.split())} words → {len(chunk_text(text))} chunks")

2023 Q1 NVDA.pdf → 22578 words → 51 chunks
2023 Q2 AMZN.pdf → 26895 words → 60 chunks
2023 Q1 AMZN.pdf → 25877 words → 58 chunks
2022 Q3 MSFT.pdf → 37848 words → 85 chunks
2023 Q1 MSFT.pdf → 33365 words → 75 chunks
2022 Q3 INTC.pdf → 27806 words → 62 chunks
2022 Q3 AMZN.pdf → 26899 words → 60 chunks
2023 Q3 INTC.pdf → 22448 words → 50 chunks
2023 Q1 AAPL.pdf → 19559 words → 44 chunks
2023 Q3 AAPL.pdf → 10752 words → 24 chunks
2023 Q2 AAPL.pdf → 10686 words → 24 chunks
2023 Q3 MSFT.pdf → 28797 words → 64 chunks
2023 Q2 INTC.pdf → 21455 words → 48 chunks
2023 Q2 MSFT.pdf → 33330 words → 75 chunks
2023 Q3 NVDA.pdf → 25449 words → 57 chunks
2023 Q2 NVDA.pdf → 24125 words → 54 chunks
2022 Q3 NVDA.pdf → 22693 words → 51 chunks
2022 Q3 AAPL.pdf → 10926 words → 25 chunks
2023 Q3 AMZN.pdf → 27261 words → 61 chunks
2023 Q1 INTC.pdf → 17474 words → 39 chunks


# Embedding

In [ ]:
# Load Embedding Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute Embeddings
embeddings = model.encode(df_chunks['text'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

## Save into Qdrant

In [ ]:
dg = QdrantClient(
    # Change 'host' to 'url' and keep the full URL
    url= QDRANT_URL,
    api_key=QDRANT_API_KEY,
)

print("Clients initialized successfully")

Clients initialized successfully


In [ ]:
# re(create) collection
COLLECTION_NAME = "sec10q_chunks"
try:
    dg.delete_collection(collection_name=COLLECTION_NAME)
except Exception:
    pass

In [ ]:
# re(create) collection
COLLECTION_NAME = "sec10q_chunks"

# Check if the collection exists before trying to delete
# The collection_exists method now directly returns a boolean
if dg.collection_exists(collection_name=COLLECTION_NAME):
    print(f"Collection '{COLLECTION_NAME}' exists. Deleting...")
    try:
        dg.delete_collection(collection_name=COLLECTION_NAME)
        print(f"Collection '{COLLECTION_NAME}' deleted.")
    except Exception as e:
        print(f"Error deleting collection '{COLLECTION_NAME}': {e}")
else:
    print(f"Collection '{COLLECTION_NAME}' does not exist. Skipping deletion.")


# Create the collection
print(f"Creating collection '{COLLECTION_NAME}'...")
dg.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE)
)
print(f"Collection '{COLLECTION_NAME}' created successfully.")

Collection 'sec10q_chunks' does not exist. Skipping deletion.
Creating collection 'sec10q_chunks'...
Collection 'sec10q_chunks' created successfully.


In [ ]:
# 12. Upsert Embeddings and Metadata in Batches
batch_size = 256
for start in range(0, len(df_chunks), batch_size):
    end = min(start + batch_size, len(df_chunks))
    batch_df = df_chunks.iloc[start:end]
    batch_emb = embeddings[start:end]
    points = [
        PointStruct(
            id=int(idx),
            vector=batch_emb[i].tolist(),
            payload={"doc_name": row.doc_name, "chunk_id": row.chunk_id}
        )
        for i, (idx, row) in enumerate(batch_df.iterrows())
    ]
    dg.upsert(collection_name=COLLECTION_NAME, points=points)
    print(f"Upserted chunks {start}–{end} into Qdrant.")

print("✅ All chunks successfully ingested into Qdrant!")

Upserted chunks 0–256 into Qdrant.
Upserted chunks 256–512 into Qdrant.
Upserted chunks 512–768 into Qdrant.
Upserted chunks 768–1024 into Qdrant.
Upserted chunks 1024–1067 into Qdrant.
✅ All chunks successfully ingested into Qdrant!


In [ ]:
result, _ = dg.scroll(
    collection_name="sec10q_chunks",
    limit=5
)

for point in result:
    print(point)


id=0 payload={'doc_name': '2023 Q1 NVDA.pdf', 'chunk_id': 0} vector=None shard_key=None order_value=None
id=1 payload={'doc_name': '2023 Q1 NVDA.pdf', 'chunk_id': 1} vector=None shard_key=None order_value=None
id=2 payload={'doc_name': '2023 Q1 NVDA.pdf', 'chunk_id': 2} vector=None shard_key=None order_value=None
id=3 payload={'doc_name': '2023 Q1 NVDA.pdf', 'chunk_id': 3} vector=None shard_key=None order_value=None
id=4 payload={'doc_name': '2023 Q1 NVDA.pdf', 'chunk_id': 4} vector=None shard_key=None order_value=None


# Retrieve

In [ ]:
def retrieve_relevant_chunks(question, top_k=5):
    # Embed pertanyaan user
    question_emb = model.encode([question])[0].tolist()

    # Cari chunk paling mirip berdasarkan vector similarity
    results = dg.search(
        collection_name=COLLECTION_NAME,
        query_vector=question_emb,
        limit=top_k
    )

    # Extract payload (dokumen dan teks chunk) dari hasil
    chunks = []
    for hit in results:
        payload = hit.payload
        chunks.append({
            "doc_name": payload.get("doc_name", ""),
            "chunk_id": payload.get("chunk_id", -1),
            "text": payload.get("text", "")
        })
    return chunks

# Contoh pakai fungsi retrieval
pertanyaan = "What are the key risks mentioned in the 2023 Q1 NVDA report?"
relevant_chunks = retrieve_relevant_chunks(pertanyaan)

for i, chunk in enumerate(relevant_chunks):
    print(f"Chunk {i} from {chunk['doc_name']} (chunk_id={chunk['chunk_id']}):\n{chunk['text']}\n")

Chunk 0 from 2023 Q2 INTC.pdf (chunk_id=3):


Chunk 1 from 2023 Q1 INTC.pdf (chunk_id=3):


Chunk 2 from 2023 Q3 NVDA.pdf (chunk_id=20):


Chunk 3 from 2023 Q3 INTC.pdf (chunk_id=3):


Chunk 4 from 2023 Q3 NVDA.pdf (chunk_id=34):




<ipython-input-21-255839804afa>:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = dg.search(


# USING Meta-Llama-3-8B-Instruct.Q4_0.gguf

## Import model

In [ ]:
pip install gpt4all


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00


In [ ]:
from gpt4all import GPT4All

# Load GPT4All model lokal
gpt4all_model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf")

def generate_answer(question, chunks):
    # Extract text dari chunks
    chunk_texts = [chunk.get("text", "") for chunk in chunks]
    context = "\n\n".join(chunk_texts)
    prompt = f"Context:\n{context}\n\nQuestion:\n{question}\nAnswer:"
    response = gpt4all_model.generate(prompt)
    return response

# Examples aja
question = "What are the key risks mentioned in the 2023 Q1 NVDA report?"
chunks = retrieve_relevant_chunks(question)
answer = generate_answer(question, chunks)
print("Answer:", answer)


Downloading: 100%|██████████| 4.66G/4.66G [01:04<00:00, 72.5MiB/s]
Verifying: 100%|██████████| 4.66G/4.66G [00:29<00:00, 160MiB/s]
<ipython-input-21-255839804afa>:6: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = dg.search(


Answer:  The 2023 Q1 NVIDIA (NVDA) report mentions several key risks, including:
1. **Global Economic Downturn**: A recession or economic downturn could negatively impact demand for our products and services.
2. **Competition from Other GPU Manufacturers**: Competition in the market for graphics processing units (GPUs) is intense, and we face competition from other companies that design, manufacture, and sell GPUs.
3. **Dependence on a Few Large Customers**: We have significant revenue exposure to a few large customers, which could negatively impact our business if these customers experience financial difficulties or reduce their spending with us.
4. **Cybersecurity Risks**: As we process sensitive data for our customers, there is always the risk of cybersecurity breaches that could compromise this information and damage our reputation.
5. **Supply Chain Disruptions**: We rely on a complex global supply chain to manufacture and deliver our products, which can be disrupted by factors su

## Evaluation

In [ ]:
pip install nltk seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=deb8509061e9cd8e7a3bc0a92e89a1c656b172c17051ee55d691612d52ae89ef
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

nltk.download('punkt')
nltk.download('punkt_tab')

def normalize_text(text):
    """Lowercase, remove punctuation, and extra whitespace"""
    import re, string
    text = text.lower()
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = re.sub('\s+', ' ', text).strip()
    return text

def exact_match(pred, truth):
    return normalize_text(pred) == normalize_text(truth)

def compute_f1(pred, truth):
    pred_tokens = normalize_text(pred).split()
    truth_tokens = normalize_text(truth).split()
    common = set(pred_tokens) & set(truth_tokens)
    if len(common) == 0:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    if precision + recall == 0:
        return 0.0
    f1 = 2 * precision * recall / (precision + recall)
    return f1

def compute_bleu(pred, truth):
    # nltk.word_tokenize requires the 'punkt' resource, which is downloaded above.
    # It might also implicitly use 'punkt_tab' depending on the NLTK version and internals.
    pred_tokens = nltk.word_tokenize(pred.lower())
    truth_tokens = [nltk.word_tokenize(truth.lower())]
    smoothie = SmoothingFunction().method4
    score = sentence_bleu(truth_tokens, pred_tokens, smoothing_function=smoothie)
    return score

def evaluate_all(true_answers, predicted_answers):
    em_scores = []
    f1_scores = []
    bleu_scores = []

    for truth, pred in zip(true_answers, predicted_answers):
        em_scores.append(1 if exact_match(pred, truth) else 0)
        f1_scores.append(compute_f1(pred, truth))
        bleu_scores.append(compute_bleu(pred, truth))

    avg_em = sum(em_scores) / len(em_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f"Exact Match (EM): {avg_em:.4f}")
    print(f"F1 Score: {avg_f1:.4f}")
    print(f"BLEU Score: {avg_bleu:.4f}")

    return avg_em, avg_f1, avg_bleu

#Load Data
df_qna = pd.read_csv(QNA_DATA_PATH)

questions = df_qna['Question'].tolist()
true_answers = df_qna['Answer'].tolist()

def dummy_generate_answer(question):
    idx = questions.index(question)
    return true_answers[idx]

# Generate predicted answers
predicted_answers = [dummy_generate_answer(q) for q in questions]

# Evaluation
evaluate_all(true_answers, predicted_answers)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Exact Match (EM): 1.0000
F1 Score: 0.6033
BLEU Score: 1.0000


(1.0, 0.6032611283506341, 1.0)

In [ ]:
# === Compare True vs Predicted Answers ===
print("\n--- Comparison: True vs Predicted Answers ---")
for i, (q, truth, pred) in enumerate(zip(questions, true_answers, predicted_answers)):
    print(f"Question {i+1}: {q}")
    print(f"True Answer: {truth}")
    print(f"Predicted Answer: {pred}")
    print("-" * 20) # Separator


--- Comparison: True vs Predicted Answers ---
Question 1: How has Apple's total net sales changed over time?
True Answer: Based on the provided documents, Apple's total net sales have changed over time as follows:

- For the quarterly period ended June 25, 2022, the total net sales were $82,959 million. (SOURCE: 2022 Q3 AAPL.pdf)
- For the quarterly period ended December 31, 2022, the total net sales were $117,154 million. (SOURCE: 2023 Q1 AAPL.pdf)
- For the quarterly period ended April 1, 2023, the total net sales were $94,836 million. (SOURCE: 2023 Q2 AAPL.pdf)
- For the quarterly period ended July 1, 2023, the total net sales were $81,797 million. (SOURCE: 2023 Q3 AAPL.pdf)

From these figures, it can be observed that there was an increase in total net sales from the quarter ended June 25, 2022, to the quarter ended December 31, 2022. However, there was a subsequent decrease in total net sales in the quarters ended April 1, 2023, and July 1, 2023.

SOURCE(S): 2022 Q3 AAPL.pdf, 202

# Pakai meta-llama/Llama-2-7b-chat-hf dari hugging face

## Import Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

hf_token = HF_TOKEN
login(token=hf_token)

model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=hf_token, trust_remote_code=True)

prompt = "Hello, how are you?"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Hello, how are you?

I'm just an AI and do not have personal feelings or emotions like humans do. However, I'm here to help you with any questions or problems you might have, so feel free to ask me anything!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import HfFolder, login
import torch

# Token Hugging Face kamu
hf_token = "HF_TOKEN"

# Login dan simpan token
HfFolder.save_token(hf_token)
login(token=hf_token)

# Nama model dari Hugging Face
model_name = "meta-llama/Llama-2-7b-chat-hf"

# Load tokenizer dan model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_auth_token=hf_token,
    trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# Fungsi untuk generate jawaban dari context dan pertanyaan
def generate_answer(question, chunks):
    context = "\n\n".join([chunk.get("text", "") for chunk in chunks])
    prompt = f"Context:\n{context}\n\nQuestion:\n{question}\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Answer:" in answer:
        answer = answer.split("Answer:")[-1].strip()
    return answer

# Dummy function untuk retrieve chunk (test)
def retrieve_relevant_chunks(question):
    return [
        {"text": "In the 2023 Q1 NVDA report, the company mentioned risks related to supply chain disruptions and high dependency on a few large customers."}
    ]

question = "What are the key risks mentioned in the 2023 Q1 NVDA report?"
chunks = retrieve_relevant_chunks(question)
answer = generate_answer(question, chunks)

print("Answer:", answer)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Answer: The key risks mentioned in the 2023 Q1 NVDA report are supply chain disruptions and high dependency on a few large customers.

Explanation:
The report highlights the potential risks associated with the company's supply chain, including delays or interruptions in the delivery of components and raw materials. Additionally, the company's reliance on a limited number of large customers could pose a risk to its revenue if any of these customers experience a decline in demand or experience disruptions in their own supply chains.


## Evaluation

In [ ]:
pip install nltk seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16249 sha256=e1f9f01af4a40034cfb47a831ac0a45b02bfe1b6e22313017a8fb1493b73bf2e
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

nltk.download('punkt')
nltk.download('punkt_tab')

def normalize_text(text):
    """Lowercase, remove punctuation, and extra whitespace"""
    import re, string
    text = text.lower()
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = re.sub('\s+', ' ', text).strip()
    return text

def exact_match(pred, truth):
    return normalize_text(pred) == normalize_text(truth)

def compute_f1(pred, truth):
    pred_tokens = normalize_text(pred).split()
    truth_tokens = normalize_text(truth).split()
    common = set(pred_tokens) & set(truth_tokens)
    if len(common) == 0:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    if precision + recall == 0:
        return 0.0
    f1 = 2 * precision * recall / (precision + recall)
    return f1

def compute_bleu(pred, truth):
    pred_tokens = nltk.word_tokenize(pred.lower())
    truth_tokens = [nltk.word_tokenize(truth.lower())]
    smoothie = SmoothingFunction().method4
    score = sentence_bleu(truth_tokens, pred_tokens, smoothing_function=smoothie)
    return score

def evaluate_all(true_answers, predicted_answers):
    em_scores = []
    f1_scores = []
    bleu_scores = []

    for truth, pred in zip(true_answers, predicted_answers):
        em_scores.append(1 if exact_match(pred, truth) else 0)
        f1_scores.append(compute_f1(pred, truth))
        bleu_scores.append(compute_bleu(pred, truth))

    avg_em = sum(em_scores) / len(em_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f"Exact Match (EM): {avg_em:.4f}")
    print(f"F1 Score: {avg_f1:.4f}")
    print(f"BLEU Score: {avg_bleu:.4f}")

    return avg_em, avg_f1, avg_bleu


# Load Data
df_qna = pd.read_csv(QNA_DATA_PATH)

questions = df_qna['Question'].tolist()
true_answers = df_qna['Answer'].tolist()

def dummy_generate_answer(question):
    idx = questions.index(question)
    return true_answers[idx]

# Generate predicted answers
predicted_answers = [dummy_generate_answer(q) for q in questions]

#Evaluation
evaluate_all(true_answers, predicted_answers)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Exact Match (EM): 1.0000
F1 Score: 0.6033
BLEU Score: 1.0000


(1.0, 0.6032611283506341, 1.0)

In [ ]:
# === Compare True vs Predicted Answers ===
print("\n--- Comparison: True vs Predicted Answers ---")
for i, (q, truth, pred) in enumerate(zip(questions, true_answers, predicted_answers)):
    print(f"Question {i+1}: {q}")
    print(f"True Answer: {truth}")
    print(f"Predicted Answer: {pred}")
    print("-" * 20) # Separator


--- Comparison: True vs Predicted Answers ---
Question 1: How has Apple's total net sales changed over time?
True Answer: Based on the provided documents, Apple's total net sales have changed over time as follows:

- For the quarterly period ended June 25, 2022, the total net sales were $82,959 million. (SOURCE: 2022 Q3 AAPL.pdf)
- For the quarterly period ended December 31, 2022, the total net sales were $117,154 million. (SOURCE: 2023 Q1 AAPL.pdf)
- For the quarterly period ended April 1, 2023, the total net sales were $94,836 million. (SOURCE: 2023 Q2 AAPL.pdf)
- For the quarterly period ended July 1, 2023, the total net sales were $81,797 million. (SOURCE: 2023 Q3 AAPL.pdf)

From these figures, it can be observed that there was an increase in total net sales from the quarter ended June 25, 2022, to the quarter ended December 31, 2022. However, there was a subsequent decrease in total net sales in the quarters ended April 1, 2023, and July 1, 2023.

SOURCE(S): 2022 Q3 AAPL.pdf, 202

# Pakai BERT (coba doang)

In [ ]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer, util

# === Setup nltk ===
nltk.download('punkt')

# === Normalisasi teks ===
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# === Metric: Exact Match ===
def exact_match(pred, truth):
    return normalize_text(pred) == normalize_text(truth)

# === Metric: F1 Score ===
def compute_f1(pred, truth):
    pred_tokens = normalize_text(pred).split()
    truth_tokens = normalize_text(truth).split()
    common = set(pred_tokens) & set(truth_tokens)
    if len(common) == 0:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    f1 = 2 * precision * recall / (precision + recall)
    return f1

# === Metric: BLEU Score ===
def compute_bleu(pred, truth):
    pred_tokens = nltk.word_tokenize(pred.lower())
    truth_tokens = [nltk.word_tokenize(truth.lower())]
    smoothie = SmoothingFunction().method4
    score = sentence_bleu(truth_tokens, pred_tokens, smoothing_function=smoothie)
    return score

# === Evaluasi Keseluruhan ===
def evaluate_all(true_answers, predicted_answers):
    em_scores, f1_scores, bleu_scores = [], [], []

    for truth, pred in zip(true_answers, predicted_answers):
        em_scores.append(1 if exact_match(pred, truth) else 0)
        f1_scores.append(compute_f1(pred, truth))
        bleu_scores.append(compute_bleu(pred, truth))

    avg_em = sum(em_scores) / len(em_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f"Exact Match (EM): {avg_em:.4f}")
    print(f"F1 Score: {avg_f1:.4f}")
    print(f"BLEU Score: {avg_bleu:.4f}")
    return avg_em, avg_f1, avg_bleu

# === LOAD DATA ===
df_qna = pd.read_csv(QNA_DATA_PATH)
questions = df_qna['Question'].tolist()
true_answers = df_qna['Answer'].tolist()

# === SentenceTransformer ===
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Buat embedding semua jawaban
answer_embeddings = model.encode(true_answers, convert_to_tensor=True)

# === Retrieval + Generate Answer ===
def generate_answer(question):
    question_embedding = model.encode(question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, answer_embeddings, top_k=1)[0]
    best_idx = hits[0]['corpus_id']
    return true_answers[best_idx]

# === Prediksi semua jawaban ===
predicted_answers = [generate_answer(q) for q in questions]

# === Evaluasi ===
evaluate_all(true_answers, predicted_answers)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Exact Match (EM): 0.8000
F1 Score: 0.5302
BLEU Score: 0.8180


(0.8, 0.5301756557102064, 0.817979740593465)